In [1]:
import pandas as pd
import pandana as pdna
import time
import os
import json
import urllib
from shapely.geometry import Point, shape

import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network

import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [2]:
def rename_nodes(nodes_df, edges_df, node_id_name, to_name, from_name):
    nodes_df['old_node_id']=nodes_df[node_id_name].copy()
    nodes_df['id_int']=range(1, len(nodes_df)+1)
    node_name_map={nodes_df.iloc[i]['old_node_id']: i+1 for i in range(len(nodes_df))}
    edges_df['from_int']=edges_df.apply(lambda row: node_name_map[row[from_name]], axis=1)
    edges_df['to_int']=edges_df.apply(lambda row: node_name_map[row[to_name]], axis=1)
    return nodes_df, edges_df, node_name_map

In [3]:
city='Detroit'

configs=json.load(open('configs.json'))
city_configs=configs[city]

# OSM Data

In [4]:
bbox=city_configs['bboxes']['amenities']

In [5]:
nodes, edges = ua.osm.load.ua_network_from_bbox(bbox=tuple(bbox),
                                                remove_lcn=True)

all_nodes_included=set(list(edges['from'].values)+list(edges['to'].values))
nodes=nodes.loc[nodes['id'].isin(all_nodes_included)]
ua.osm.network.create_osm_net(osm_edges=edges,
                              osm_nodes=nodes,
                              travel_speed_mph=3)

Requesting network data within bounding box from Overpass API in 1 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["pedestrians"!~"no"](42.31400000,-83.09600000,42.35000000,-83.04000000);>;);out;'}"
Downloaded 4,004.1KB from www.overpass-api.de in 0.76 seconds
Downloaded OSM network data within bounding box from Overpass API in 1 request(s) and 1.03 seconds
Returning OSM data with 23,278 nodes and 7,292 ways...
Edge node pairs completed. Took 22.17 seconds
Returning processed graph with 10,909 nodes and 34,690 edges...
Completed OSM data download and Pandana node and edge table creation in 24.25 seconds
checking for low connectivity nodes...
23 out of 10,909 nodes (0.21 percent of total) were identified as having low connectivity and have been removed.
Completed OSM data download and graph node and edge table creation i

In [6]:
# edges=pd.read_csv('./'+city+'/data/osm_ped_network_edges.csv')
# nodes=pd.read_csv('./'+city+'/data/osm_ped_network_nodes.csv')


# GFTS Data

In [7]:
# GTFS Data
if 'feed_dicts' in city_configs:
    # if gtfs data available, create and save the combined network
    feeds.add_feed(add_dict=city_configs['feed_dicts'])

    os.chdir(city)
#     gtfsfeeds.download()

    validation = True
    verbose = True
    remove_stops_outsidebbox = True
    append_definitions = True

    loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(gtfsfeed_path=None,
                                           validation=validation,
                                           verbose=verbose,
                                           bbox=tuple(bbox),
                                           remove_stops_outsidebbox=remove_stops_outsidebbox,
                                           append_definitions=append_definitions)

    ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='monday',
                                   timerange=['07:00:00', '10:00:00'],
                                   calendar_dates_lookup=None)

    urbanaccess_net = ua.network.ua_network

    #integrate the networks
    ua.network.integrate_network(urbanaccess_network=urbanaccess_net,
                             headways=False,
                             urbanaccess_gtfsfeeds_df=loaded_feeds)
    urbanaccess_net.net_nodes.to_csv('./data/access_network_nodes.csv')
    urbanaccess_net.net_edges.to_csv('./data/access_network_edges.csv')
else:
    print('Ped network only')
    # otherwise, reformat the ped network to the urbanaccess format and save that
    edges['weight']=edges.apply(lambda row: row['distance']/(5000/(60)), axis=1) # 5 km/hr
    nodes, edges, _=rename_nodes(nodes, edges, 'id', 'to', 'from')
    edges.to_csv('./'+city+'/data/access_network_edges.csv')
    nodes.to_csv('./'+city+'/data/access_network_nodes.csv')
    

Added 2 feeds to gtfs_feeds: {'DDOT': 'https://data.detroitmi.gov/download/y62d-bvsz/application%2Fzip', 'People Mover': 'https://data.detroitmi.gov/api/views/4bkd-9u3w/files/c32fecf1-d8cc-4e53-8fc1-a021ccf4d0a9?filename=dtc-gtfs.zip'}
GTFS text file header whitespace check completed. Took 1.58 seconds
--------------------------------
Processing GTFS feed: DDOT
The unique agency id: detroit_department_of_transportation was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.02 seconds
Unique GTFS feed id operation complete. Took 0.01 seconds
Records: {}
     stop_id stop_code                     stop_name  \
0         15    WYMNEB            Wyoming & Michigan   
2         22    MKMIEB               MACK & MITCHELL   
3         25    MKMLWB             MACK & MC DOUGALL   
4         32    LRCHEB                Larned & Chene   
5         33    KCBVEB  Kercheval & East Grand Blvd.   
6         34    MCKCEB         McClellan & Kerche

Appended route type to stops
Appended route type to stop_times
--------------------------------
--------------------------------
Processing GTFS feed: People Mover
The unique agency id: people_mover was generated using the name of the agency in the agency.txt file.
Unique agency id operation complete. Took 0.02 seconds
Unique GTFS feed id operation complete. Took 0.01 seconds
Records: {}
    stop_id  stop_code  platform_code  \
0     10023      117.0            NaN   
1     10024      106.0            NaN   
2     10026      105.0            NaN   
3     10028      120.0            NaN   
4     10029      119.0            NaN   
5     10031      118.0            NaN   
6     10032      101.0            NaN   
7   2505180      130.0            NaN   
8   2506704      131.0            NaN   
9   2506705      132.0            NaN   
10  2506706      133.0            NaN   
11  2506707      134.0            NaN   
12  2506708      135.0            NaN   
13  2506709      136.0            N

Added descriptive definitions to stops, routes, stop_times, and trips tables
Successfully converted ['departure_time'] to seconds past midnight and appended new columns to stop_times. Took 0.19 seconds
2 GTFS feed file(s) successfully read as dataframes:
     DDOT
     People Mover
     Took 3.89 seconds
Using calendar to extract service_ids to select trips.
3 service_ids were extracted from calendar
2,780 trip(s) 45.18 percent of 6,153 total trip records were found in calendar for GTFS feed(s): ['ddot', 'people mover']
NOTE: If you expected more trips to have been extracted and your GTFS feed(s) have a calendar_dates file, consider utilizing the calendar_dates_lookup parameter in order to add additional trips based on information inside of calendar_dates. This should only be done if you know the corresponding GTFS feed is using calendar_dates instead of calendar to specify service_ids. When in doubt do not use the calendar_dates_lookup parameter.
2,780 of 6,153 total trips were extrac

In [8]:
#os.getcwd()
#os.chdir('../')